In [1]:
import pandas as pd

# Assuming the dataset is in CSV format. Replace 'path_to_dataset.csv' with the actual file path
df = pd.read_csv('drugLibTrain_raw.tsv', delimiter='\t')
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")


Number of rows: 3107
Number of columns: 9


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import HashingVectorizer


# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

df = df.dropna()
# Create TF-IDF vectors for each free-text column
# tfidf_benefits = tfidf_vectorizer.fit_transform(df['benefitsReview'])
# tfidf_sideEffects = tfidf_vectorizer.fit_transform(df['sideEffectsReview'])
# tfidf_comments = tfidf_vectorizer.fit_transform(df['commentsReview'])


count_vectorizer = CountVectorizer()

tfidf_benefits = count_vectorizer.fit_transform(df['benefitsReview'])
tfidf_sideEffects = count_vectorizer.fit_transform(df['sideEffectsReview'])
tfidf_comments = count_vectorizer.fit_transform(df['commentsReview'])


# hashing_vectorizer = HashingVectorizer()

# tfidf_benefits = hashing_vectorizer.fit_transform(df['benefitsReview'])
# tfidf_sideEffects = hashing_vectorizer.fit_transform(df['sideEffectsReview'])
# tfidf_comments = hashing_vectorizer.fit_transform(df['commentsReview'])



# # Tokenize the text data
# tfidf_benefits = df['benefitsReview'].apply(lambda x: x.split())
# tfidf_sideEffects = df['sideEffectsReview'].apply(lambda x: x.split())
# tfidf_comments = df['commentsReview'].apply(lambda x: x.split())

# # Train Word2Vec models
# tfidf_benefits = Word2Vec(tfidf_benefits, min_count=1)
# tfidf_sideEffects = Word2Vec(tfidf_sideEffects, min_count=1)
# tfidf_comments = Word2Vec(tfidf_comments, min_count=1)

# # Convert the text data into word embeddings
# tfidf_benefits = tfidf_benefits.wv[tfidf_benefits]
# tfidf_sideEffects = tfidf_sideEffects.wv[tfidf_sideEffects]
# tfidf_comments = tfidf_comments.wv[tfidf_comments]


In [22]:
from scipy.sparse import hstack

# Combine the TF-IDF vectors into one matrix
tfidf_combined = hstack([tfidf_benefits, tfidf_sideEffects, tfidf_comments])


In [23]:
from sklearn.model_selection import train_test_split

# Features
X = tfidf_combined

# Labels
y_rating = df['rating']
y_effectiveness = df['effectiveness']
y_sideEffects = df['sideEffects']

# Split the data
X_train, X_test, y_train_rating, y_test_rating = train_test_split(X, y_rating, test_size=0.2, random_state=42)
X_train, X_test, y_train_effectiveness, y_test_effectiveness = train_test_split(X, y_effectiveness, test_size=0.2, random_state=42)
X_train, X_test, y_train_sideEffects, y_test_sideEffects = train_test_split(X, y_sideEffects, test_size=0.2, random_state=42)


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score

# Assuming 'tfidf_combined' is your combined TF-IDF matrix and
# 'df['rating']', 'df['effectiveness']', 'df['sideEffects']' are your labels

# Split the data for rating prediction
X_train_rating, X_test_rating, y_train_rating, y_test_rating = train_test_split(
    tfidf_combined, df['rating'], test_size=0.2, random_state=42)

# Split the data for effectiveness prediction
X_train_effectiveness, X_test_effectiveness, y_train_effectiveness, y_test_effectiveness = train_test_split(
    tfidf_combined, df['effectiveness'], test_size=0.2, random_state=42)

# Split the data for sideEffect prediction
X_train_sideEffect, X_test_sideEffect, y_train_sideEffect, y_test_sideEffect = train_test_split(
    tfidf_combined, df['sideEffects'], test_size=0.2, random_state=42)

# Initialize models
rating_model = RandomForestRegressor()
effectiveness_model = RandomForestClassifier()
sideEffect_model = RandomForestClassifier()

# Train models
rating_model.fit(X_train_rating, y_train_rating)
effectiveness_model.fit(X_train_effectiveness, y_train_effectiveness)
sideEffect_model.fit(X_train_sideEffect, y_train_sideEffect)

# Predict
rating_predictions = rating_model.predict(X_test_rating)
effectiveness_predictions = effectiveness_model.predict(X_test_effectiveness)
sideEffect_predictions = sideEffect_model.predict(X_test_sideEffect)

# Evaluate
rating_mse = mean_squared_error(y_test_rating, rating_predictions)
effectiveness_accuracy = accuracy_score(y_test_effectiveness, effectiveness_predictions)
sideEffect_accuracy = accuracy_score(y_test_sideEffect, sideEffect_predictions)

# Print out the performance
print(f'Rating MSE: {rating_mse}')
print(f'Effectiveness Accuracy: {effectiveness_accuracy}')
print(f'SideEffect Accuracy: {sideEffect_accuracy}')


Rating MSE: 5.921549169435217
Effectiveness Accuracy: 0.45016611295681064
SideEffect Accuracy: 0.5365448504983389


In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load the dataset
# df = pd.read_csv('path_to_your_dataset.csv')
df = pd.read_csv('drugLibTrain_raw.tsv', delimiter='\t')
# Handle missing values
df = df.dropna()

# Convert categorical labels to numerical
le = LabelEncoder()
df['effectiveness'] = le.fit_transform(df['effectiveness'])
df['sideEffects'] = le.fit_transform(df['sideEffects'])

# Generate TF-IDF vectors
# tfidf_vectorizer = TfidfVectorizer()
# tfidf_combined = tfidf_vectorizer.fit_transform(df['benefitsReview'] + ' ' + df['sideEffectsReview'] + ' ' + df['commentsReview'])

# count_vectorizer = CountVectorizer()
# tfidf_combined = count_vectorizer.fit_transform(df['benefitsReview'] + ' ' + df['sideEffectsReview'] + ' ' + df['commentsReview'])

hashing_vectorizer = HashingVectorizer()
tfidf_combined = hashing_vectorizer.fit_transform(df['benefitsReview'] + ' ' + df['sideEffectsReview'] + ' ' + df['commentsReview'])


# combined_text = (df['benefitsReview'] + ' ' + df['sideEffectsReview'] + ' ' + df['commentsReview']).apply(lambda x: x.split())
# w2v_model = Word2Vec(combined_text, min_count=1)
# tfidf_combined = combined_text.apply(lambda x: [w2v_model.wv[word] for word in x])


# Split the data
X_train, X_test, y_train_rating, y_test_rating = train_test_split(tfidf_combined, df['rating'], test_size=0.2, random_state=42)
X_train, X_test, y_train_effectiveness, y_test_effectiveness = train_test_split(tfidf_combined, df['effectiveness'], test_size=0.2, random_state=42)
X_train, X_test, y_train_sideEffect, y_test_sideEffect = train_test_split(tfidf_combined, df['sideEffects'], test_size=0.2, random_state=42)

# Initialize models
rating_model = RandomForestRegressor(n_estimators=100, max_depth=None, random_state=0)
effectiveness_model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=0)
sideEffect_model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=0)

# Train models
rating_model.fit(X_train, y_train_rating)
effectiveness_model.fit(X_train, y_train_effectiveness)
sideEffect_model.fit(X_train, y_train_sideEffect)

# Predict
rating_predictions = rating_model.predict(X_test)
effectiveness_predictions = effectiveness_model.predict(X_test)
sideEffect_predictions = sideEffect_model.predict(X_test)

# Evaluate
rating_mse = mean_squared_error(y_test_rating, rating_predictions)
effectiveness_accuracy = accuracy_score(y_test_effectiveness, effectiveness_predictions)
sideEffect_accuracy = accuracy_score(y_test_sideEffect, sideEffect_predictions)

# Print out the performance.00
print(f'Rating MSE: {rating_mse}')
print(f'Effectiveness Accuracy: {effectiveness_accuracy}')
print(f'SideEffect Accuracy: {sideEffect_accuracy}')


Rating MSE: 6.924594518272425
Effectiveness Accuracy: 0.4418604651162791
SideEffect Accuracy: 0.45514950166112955


In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Combine all the TF-IDF dataframes
# df_tfidf = pd.concat([df_tfidf_benefits, df_tfidf_sideEffects, df_tfidf_comments], axis=1)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_combined, df['rating'], test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
# clf = RandomForestClassifier(n_estimators=100, random_state=42)
# clf = MultinomialNB()
clf = SVC()


# Fit the model
clf.fit(X_train, y_train)

# Predict the ratings
y_pred = clf.predict(X_test)


# Calculate metrics for rating prediction
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Confusion Matrix: \n{conf_matrix}")

Accuracy: 0.3023255813953488
Precision: 0.2614199426699427
Recall: 0.3023255813953488
F1 Score: 0.21762052384882627
Confusion Matrix: 
[[ 17   0   0   0   0   0   0   8   0  27]
 [  5   0   0   0   0   0   0   5   0  11]
 [  3   0   0   0   0   0   0  16   2  11]
 [  2   0   0   0   0   0   0   9   0  12]
 [  3   0   0   0   0   0   0  13   2  14]
 [  1   0   0   0   0   1   1  18   1  18]
 [  2   0   0   0   0   0   0  26   4  26]
 [  1   0   0   0   0   0   0  41   7  68]
 [  3   0   0   0   0   0   0  24   6  48]
 [  2   0   0   0   0   0   0  25   2 117]]


/Users/vickyprince/opt/anaconda3/envs/myFirstEnv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Combine all the TF-IDF dataframes
# df_tfidf = pd.concat([df_tfidf_benefits, df_tfidf_sideEffects, df_tfidf_comments], axis=1)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_combined, df['effectiveness'], test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
# clf = RandomForestClassifier(n_estimators=100, random_state=42)
# clf = MultinomialNB()
clf = SVC()

# Fit the model
clf.fit(X_train, y_train)

# Predict the ratings
y_pred = clf.predict(X_test)


# Calculate metrics for rating prediction
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Confusion Matrix: \n{conf_matrix}")

Accuracy: 0.45348837209302323
Precision: 0.4443502685981276
Recall: 0.45348837209302323
F1 Score: 0.35305486956488386
Confusion Matrix: 
[[ 38 151   0   0   0]
 [ 26 233   0   0   0]
 [  2  41   1   0   0]
 [  7  22   0   1   0]
 [ 25  55   0   0   0]]


/Users/vickyprince/opt/anaconda3/envs/myFirstEnv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Combine all the TF-IDF dataframes
# df_tfidf = pd.concat([df_tfidf_benefits, df_tfidf_sideEffects, df_tfidf_comments], axis=1)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_combined, df['sideEffects'], test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
# clf = RandomForestClassifier(n_estimators=100, random_state=42)
# clf = MultinomialNB()
clf = SVC()

# Fit the model
clf.fit(X_train, y_train)

# Predict the ratings
y_pred = clf.predict(X_test)


# Calculate metrics for rating prediction
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Confusion Matrix: \n{conf_matrix}")

Accuracy: 0.5083056478405316
Precision: 0.5314108629572397
Recall: 0.5083056478405316
F1 Score: 0.43132112140242873
Confusion Matrix: 
[[  0  20   3  10   0]
 [  0 182   5  22   0]
 [  0  98  10  10   0]
 [  0  54   3 109   0]
 [  0  50  10  11   5]]


/Users/vickyprince/opt/anaconda3/envs/myFirstEnv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
